In [1]:
using LinearAlgebra, Random, Statistics

In [2]:
# generate random data matrix
n,d = 6,4
X = randn(n,d)

# optional: give it linearly dependent columns
X[:,3] = X[:,2]

6-element Array{Float64,1}:
  0.5136788312231796
 -0.12787754532839718
 -0.4834037745160183
 -0.7915295000745926
 -1.0908843814045057
  1.5025523054723697

In [3]:
U,σ,V = svd(X)

SVD{Float64,Float64,Array{Float64,2}}
U factor:
6×4 Array{Float64,2}:
 -0.568163   -0.171815   0.0952828    0.73464
 -0.314689   -0.347533   0.217571    -0.587069
 -0.132778   -0.412281  -0.00813318   0.0667861
 -0.0664736  -0.552727  -0.745321    -0.0872516
  0.647021   -0.117238  -0.222224     0.304362
 -0.370777    0.600393  -0.581921    -0.104619
singular values:
4-element Array{Float64,1}:
 4.262098287410943
 2.5911493509088843
 1.5443517284792978
 1.1521074635733857e-16
Vt factor:
4×4 Array{Float64,2}:
 -0.86831   -0.327948   -0.327948   -0.17589
 -0.451009   0.626362    0.626362   -0.109233
  0.206466  -0.0109745  -0.0109745  -0.978331
  0.0       -0.707107    0.707107   -2.498e-16

In [4]:
U'*U

4×4 Array{Float64,2}:
  1.0          -1.01315e-16   2.26706e-17  -1.45177e-16
 -1.01315e-16   1.0          -2.33022e-16   8.99258e-17
  2.26706e-17  -2.33022e-16   1.0           2.57133e-17
 -1.45177e-16   8.99258e-17   2.57133e-17   1.0

In [5]:
V'*V

4×4 Array{Float64,2}:
  1.0          1.46294e-16  -7.2519e-17   -1.03137e-16
  1.46294e-16  1.0           3.56706e-16   2.29154e-16
 -7.2519e-17   3.56706e-16   1.0           1.04408e-16
 -1.03137e-16  2.29154e-16   1.04408e-16   1.0

In [6]:
σ

4-element Array{Float64,1}:
 4.262098287410943
 2.5911493509088843
 1.5443517284792978
 1.1521074635733857e-16

In [7]:
# if we have a linearly dependent column, 
# decomposition is just as good if we ignore the 0 in sigma and reduce r by 1
println("Error of full rank svd: ", norm(X - U*diagm(σ)*V'))
for k=3:-1:1
    println("Error of rank $k approximation: ", norm(X - U[:,1:k]*diagm(σ[1:k])*(V[:,1:k])'))
end

Error of full rank svd: 2.3529297095611565e-15
Error of rank 3 approximation: 2.3358423603593286e-15
Error of rank 2 approximation: 1.5443517284792974
Error of rank 1 approximation: 3.0164676726218245


In [8]:
# form data from noisy linear model
w♮ = randn(d)
y = X*w♮ + .1*randn(n);

In [9]:
# solve least squares problem to estimate w
w = V*diagm(σ.^(-1))*U'*y

4-element Array{Float64,1}:
 -1.5154000495613071
 -9.0444086867765e13
  9.0444086867763e13
  2.2884718050690585

In [10]:
# use rank k approximation to design matrix X
# k=4 is full rank
# when design matrix X has rank 3, k=3 gives 0 error approximation
# while k=2 results in some loss of accuracy - but not much!
k = 3
w = V[:,1:k]*diagm(σ[1:k].^(-1))*(U[:,1:k])'*y

4-element Array{Float64,1}:
 -1.5154000495613071
 -0.4085969926968041
 -0.4085969926968038
  2.320423060281874

In [11]:
# how good is our estimate of w?
norm(w - w♮) / norm(w♮)

0.28307313588650584

In [12]:
# compute mean square error
mean((y - X*w).^2)

0.002292939150367754

In [13]:
# let's use the shorthand
# backslash finds least norm solution to normal eqns
# using SVD when design matrix X is rank deficient
w_backslash = X \ y
norm(w_backslash - w)

1.1775693440128313e-15